In [127]:
import pandas as pd
from Bio import SeqIO
import subprocess

In [30]:
full_info = pd.read_csv('../1_DataSmall/fullTableInfoGff3GffRNAESwithDensity20150829.csv')
full_info.index = full_info.shortName

basepath = '/storage3/w/richard/meta2015/placeForRNASeq/'

In [215]:
print full_info.ix['Hebcy2'].intronDensityGff 
print full_info.ix['Obbri1'].intronDensityGff 

4.24
5.02


In [3]:
target = '../5_Tests/Oct15intronCompare/intronComparison.txt'

In [17]:
lines = []
with open(target) as f: 
    for line in f:
        if len(line) > 2:
            lines.append(line.rstrip())

In [18]:
result = zip(lines[::3],lines[1::3],lines[2::3])

In [22]:
shortName = lines[::3]

In [23]:
longName  = full_info.longName[shortName]

In [25]:
gc = full_info.gc[shortName]

In [27]:
fastaName = map(lambda x: x[:-3], full_info.fastaName[shortName])

In [54]:
genomesize = map(int,map(lambda x:os.path.getsize(basepath+x+"/data/"+full_info.fastaName[x][:-3])/1024./1024., shortName))

In [64]:
rna_seq_intron = map(lambda x: x.split()[0],lines[1::3])
rna_seq_intron_unique = map(lambda x: x.split()[2],lines[1::3])

In [65]:
ES_intron = map(lambda x: x.split()[0],lines[2::3])
ES_intron_unique = map(lambda x: x.split()[2],lines[2::3])

In [66]:
shared_intron = map(lambda x: x.split()[1], lines[1::3])
shared_intron_rna = map(lambda x: x.split()[1], lines[1::3])

In [70]:
testGenomeRNASeq_ES = pd.DataFrame({"shortName":shortName,
                                   "longName":longName,
                                   'gc':gc,
                                   'genomesize_Mb':genomesize,
                                   'rna_seq_intron':rna_seq_intron,
                                   'rna_seq_intron_unique':rna_seq_intron_unique,
                                   'ES_intron':ES_intron,
                                   'ES_intron_unique':ES_intron_unique,
                                   'shared_intron':shared_intron},
                                   columns = ["shortName","longName",'gc','genomesize_Mb','rna_seq_intron',
                                              'rna_seq_intron_unique','ES_intron','ES_intron_unique','shared_intron'])

In [72]:
testGenomeRNASeq_ES.head()

,shortName,longName,gc,genomesize_Mb,rna_seq_intron,rna_seq_intron_unique,ES_intron,ES_intron_unique,shared_intron
shortName,,,,,,,,,
Aspoch1,Aspoch1,Aspergillus ochraceoroseus IBT 24754 v1.0,44.2,26,52840,36554,19845,3559,16286
Cenge3,Cenge3,Cenococcum geophilum 1.58 v2.0,37.5,171,80781,57630,28373,5222,23151
Conco1,Conco1,Conidiobolus coronatus NRRL28638 v1.0,27.7,38,41143,26361,18881,4099,14782
Conli1,Conli1,Coniochaeta ligniaria CBS 111746,51.9,42,41572,27217,24418,10063,14355
Croqu1,Croqu1,Cronartium quercuum f. sp. fusiforme G11 v1.0,41.1,74,83592,64755,24342,5505,18837


In [88]:
########################################################################
########################################################################

In [85]:
####  perform tests on intron with score > 10  ####
base_command = 'perl /home/alexl/DISTR/src/pl/compare_cds_exact.pl -f1'
source = '/storage3/w/richard/meta2015/placeForRNASeq/'
target = '/home/richard/research/5_Tests/Oct15intronCompare/'
script = ''
for name in shortName:
    command = "{} {}{}/star/star_introns_GT_AG_10.gff -f2 {}{}/star/introns.gff -v -intron >{}{}.score10.txt\n".format(base_command, source,name,source, name,target,name)
    script+=command
    
with open(target+'get_intron_overlap_score_10.sh','w') as f: f.write(script)

In [89]:
source = "/home/richard/research/5_Tests/Oct15intronCompare/"
filenames = filter(lambda x: 'score10' in x, os.listdir(source))

In [93]:
result_score_10 = []
for name in shortName:
    res = [name]
    with open(source+name+'.score10.txt') as f:
        for line in f: res.append(line.rstrip())
    result_score_10.append(res)

In [95]:
def get_score10(alist):
    if len(alist) != 3:
        return [alist[0],'0 0 0 0','0 0 0 0']
    else:
        return alist

In [97]:
result_score_10 = map(get_score10,result_score_10)

In [186]:
result_score_10

[['Aspoch1',
  '22375 14680 7695 65.61 /storage3/w/richard/meta2015/placeForRNASeq/Aspoch1/star/star_introns_GT_AG_10.gff',
  '19845 14680 5165 73.97 /storage3/w/richard/meta2015/placeForRNASeq/Aspoch1/star/introns.gff'],
 ['Cenge3',
  '32091 20822 11269 64.88 /storage3/w/richard/meta2015/placeForRNASeq/Cenge3/star/star_introns_GT_AG_10.gff',
  '28373 20822 7551 73.39 /storage3/w/richard/meta2015/placeForRNASeq/Cenge3/star/introns.gff'],
 ['Conco1',
  '17943 14036 3907 78.23 /storage3/w/richard/meta2015/placeForRNASeq/Conco1/star/star_introns_GT_AG_10.gff',
  '18881 14036 4845 74.34 /storage3/w/richard/meta2015/placeForRNASeq/Conco1/star/introns.gff'],
 ['Conli1',
  '17179 12156 5023 70.76 /storage3/w/richard/meta2015/placeForRNASeq/Conli1/star/star_introns_GT_AG_10.gff',
  '24418 12156 12262 49.78 /storage3/w/richard/meta2015/placeForRNASeq/Conli1/star/introns.gff'],
 ['Croqu1',
  '39748 16163 23585 40.66 /storage3/w/richard/meta2015/placeForRNASeq/Croqu1/star/star_introns_GT_AG_10.gf

In [101]:
rna_seq_intron_score10 = map(lambda x: x.split()[0],map(lambda x: x[1],result_score_10))
rna_seq_intron_score10_unqiue = map(lambda x: x.split()[2],map(lambda x: x[1],result_score_10))

In [102]:
shared_intron_score10 = map(lambda x: x.split()[1], map(lambda x: x[1],result_score_10))

In [105]:
testGenomeRNASeq_ES['rna_seq_intron_score10'] = rna_seq_intron_score10
testGenomeRNASeq_ES['rna_seq_intron_score10_unqiue'] = rna_seq_intron_score10_unqiue
testGenomeRNASeq_ES['shared_intron_score10'] = shared_intron_score10

In [107]:
testGenomeRNASeq = pd.DataFrame({"shortName":shortName,
                                   "longName":longName,
                                   'gc':gc,
                                   'genomesize_Mb':genomesize,
                                   'rna_seq_intron':rna_seq_intron,
                                   'rna_seq_intron_unique':rna_seq_intron_unique,
                                   'ES_intron':ES_intron,
                                   'ES_intron_unique':ES_intron_unique,
                                   'shared_intron':shared_intron,
                                   'rna_seq_intron_score10':rna_seq_intron_score10,
                                   'rna_seq_intron_score10_unqiue':rna_seq_intron_score10_unqiue,
                                   'shared_intron_score10':shared_intron_score10},
                                   columns = ["shortName","longName",'gc','genomesize_Mb','rna_seq_intron',
                                              'rna_seq_intron_unique','ES_intron','ES_intron_unique','shared_intron',
                                             'rna_seq_intron_score10','rna_seq_intron_score10_unqiue','shared_intron_score10'])

In [110]:
testGenomeRNASeq.head()

,shortName,longName,gc,genomesize_Mb,rna_seq_intron,rna_seq_intron_unique,ES_intron,ES_intron_unique,shared_intron,rna_seq_intron_score10,rna_seq_intron_score10_unqiue,shared_intron_score10
shortName,,,,,,,,,,,,
Aspoch1,Aspoch1,Aspergillus ochraceoroseus IBT 24754 v1.0,44.2,26,52840,36554,19845,3559,16286,22375,7695,14680
Cenge3,Cenge3,Cenococcum geophilum 1.58 v2.0,37.5,171,80781,57630,28373,5222,23151,32091,11269,20822
Conco1,Conco1,Conidiobolus coronatus NRRL28638 v1.0,27.7,38,41143,26361,18881,4099,14782,17943,3907,14036
Conli1,Conli1,Coniochaeta ligniaria CBS 111746,51.9,42,41572,27217,24418,10063,14355,17179,5023,12156
Croqu1,Croqu1,Cronartium quercuum f. sp. fusiforme G11 v1.0,41.1,74,83592,64755,24342,5505,18837,39748,23585,16163


In [122]:
################# ########################
##########################################
########## Deal with annotation ##########
source = '/storage3/w/richard/meta2015/finalTestSet20151002/gff3/'
targetGff3 = '/storage3/w/richard/meta2015/finalTestSet20151002/gff3_intron_added/'
targetIntron = '/storage3/w/richard/meta2015/finalTestSet20151002/intron_from_gff3/'

In [125]:
## add intron to gff3 files
base_command = '/home/tool/gt/bin/gt gff3 -force -addintrons -o'
script = ''

for name in shortName:
    script += '{}  {}{}.withIntron.gff3 {}{}*.gff3\n'.format(base_command,targetGff3,name,source,name)

In [128]:
subprocess.check_call(script,shell = True)

0

In [138]:
## extract intron from annotation gff3 files
script = ''
for name in shortName:
    script += "grep -P \'\\tintron\\t\' {}{}.withIntron.gff3 > {}{}.introns.gff\n".format(targetGff3,name,targetIntron,name)

In [141]:
subprocess.check_call(script,shell = True)

0

In [142]:
## copy annotation intron to their respective folder under placeForRNA

In [148]:
target = '/storage3/w/richard/meta2015/placeForRNASeq/'
script = ''
for name in shortName:
    script += 'cp {}{}.introns.gff {}{}/star/annotation.introns.gff\n'.format(targetIntron, name, target, name)

In [150]:
subprocess.check_call(script,shell = True)

0

In [151]:
####  perform tests on rna-seq intron with annotation  ####
base_command = 'perl /home/alexl/DISTR/src/pl/compare_cds_exact.pl -f1'
source = '/storage3/w/richard/meta2015/placeForRNASeq/'
target = '/home/richard/research/5_Tests/Oct15intronCompare/intron_annotation_score10_compare/'
script = ''
for name in shortName:
    command = "{} {}{}/star/star_introns_GT_AG_10.gff -f2 {}{}/star/annotation.introns.gff -v -intron >{}{}.annotation.score10.txt\n".format(base_command, source,name,source, name,target,name)
    script+=command
    
with open(target+'get_intron_overlap_annotation_score_10.sh','w') as f: f.write(script)

In [171]:
source = "/home/richard/research/5_Tests/Oct15intronCompare/intron_annotation_score10_compare/"
filenames = filter(lambda x: 'score10' in x, os.listdir(source))

result_annotation_score_10 = []
for name in shortName:
    res = [name]
    with open(source+name+'.annotation.score10.txt') as f:
        for line in f: res.append(line.rstrip())
    result_annotation_score_10.append(res)

result_annotation_score_10 = map(get_score10,result_annotation_score_10)

In [185]:
result_annotation_score_10

[['Aspoch1',
  '22375 14680 7695 65.61 /storage3/w/richard/meta2015/placeForRNASeq/Aspoch1/star/star_introns_GT_AG_10.gff',
  '19845 14680 5165 73.97 /storage3/w/richard/meta2015/placeForRNASeq/Aspoch1/star/annotation.introns.gff'],
 ['Cenge3',
  '32091 20822 11269 64.88 /storage3/w/richard/meta2015/placeForRNASeq/Cenge3/star/star_introns_GT_AG_10.gff',
  '28373 20822 7551 73.39 /storage3/w/richard/meta2015/placeForRNASeq/Cenge3/star/annotation.introns.gff'],
 ['Conco1',
  '17943 14036 3907 78.23 /storage3/w/richard/meta2015/placeForRNASeq/Conco1/star/star_introns_GT_AG_10.gff',
  '18881 14036 4845 74.34 /storage3/w/richard/meta2015/placeForRNASeq/Conco1/star/annotation.introns.gff'],
 ['Conli1',
  '17179 12156 5023 70.76 /storage3/w/richard/meta2015/placeForRNASeq/Conli1/star/star_introns_GT_AG_10.gff',
  '24418 12156 12262 49.78 /storage3/w/richard/meta2015/placeForRNASeq/Conli1/star/annotation.introns.gff'],
 ['Croqu1',
  '39748 16163 23585 40.66 /storage3/w/richard/meta2015/placeFo

In [179]:
rna_seq_annotation_intron_score10 = map(lambda x: x.split()[0],map(lambda x: x[1],result_annotation_score_10))
rna_seq_annotation_intron_score10_unique = map(lambda x: x.split()[2],map(lambda x: x[1],result_annotation_score_10))

In [174]:
shared_annotation_intron_score10 = map(lambda x: x.split()[1], map(lambda x: x[1],result_annotation_score_10))

In [182]:
testGenomeRNASeq = pd.DataFrame({"shortName":shortName,
                                   "longName":longName,
                                   'gc':gc,
                                   'genomesize_Mb':genomesize,
                                   'rna_seq_intron':rna_seq_intron,
                                   'rna_seq_intron_unique':rna_seq_intron_unique,
                                   'ES_intron':ES_intron,
                                   'ES_intron_unique':ES_intron_unique,
                                   'shared_intron':shared_intron,
                                   'rna_seq_intron_score10':rna_seq_intron_score10,
                                   'rna_seq_intron_score10_unqiue':rna_seq_intron_score10_unqiue,
                                   'shared_intron_score10':shared_intron_score10,
                                    'rna_seq_annotation_intron_score10':rna_seq_annotation_intron_score10,
                                    'rna_seq_annotation_intron_score10_unique':rna_seq_annotation_intron_score10_unique,
                                    'shared_annotation_intron_score10':shared_annotation_intron_score10},
                                   columns = ["shortName","longName",'gc','genomesize_Mb','rna_seq_intron',
                                              'rna_seq_intron_unique','ES_intron','ES_intron_unique','shared_intron',
                                             'rna_seq_intron_score10','rna_seq_intron_score10_unqiue','shared_intron_score10',
                                             'rna_seq_annotation_intron_score10',
                                             'rna_seq_annotation_intron_score10_unique',
                                             'shared_annotation_intron_score10'])

In [184]:
testGenomeRNASeq.to_csv("../5_Tests/Oct15intronCompare/intronCompareInfo.csv")

In [206]:
## I realize that the ES prediction was actually annotation
## right now I should cp all the genemark.gtf to folder 
target = '/storage3/w/richard/meta2015/genemark_prediction/'
script = ''
for i in range(1,10):
    node = 'node00{}'.format(i)
    filenames = subprocess.Popen('ssh {} ls /scratch/fungi/ES_Run_Aug2015/'.format(node),stdout=subprocess.PIPE,shell = True).communicate()[0].split()
    for filename in filenames:
        command = 'ssh {} cp /scratch/fungi/ES_Run_Aug2015/{}/genemark.gtf {}filename.genemark.gtf\n'.format(node,filename,target)
        script+=command

In [209]:
with open('/home/richard/tempFile/cpGenemark.sh','w') as f: f.write(script)